#### Naming Convention: med_{embedding_model}_{chainning_strategy}

In [5]:
import csv
from langchain.docstore.document import Document

# Load data, store abstract text
batch_input = []
with open('../../../INLPT_data/additional_data.csv', encoding = "utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        batch_input.append(Document(page_content=row["CD"], metadata={"PMID": row["PMID"], "source": row["source"]}))

In [70]:
# batch_input = batch_input[:10]

We depend on the Tokenizer to calculate the number of tokens in one concatenatation of the metadata, then we use the chunking process and the splitter functionalities + overlaping to retrieve to split into chunks

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter # check other splitters
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("intfloat/e5-base-v2")

text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(tokenizer, chunk_size=512, chunk_overlap=100)

splitted_documents = text_splitter.split_documents(batch_input)

In [8]:
splitted_documents[34]

Document(page_content="Hong Kong wereassessed concerning their filial piety beliefs, theories of intelligence andacademic achievement. Data were analysed using structural equation modelling. Theresults indicated that different filial piety beliefs relate to students'academic achievement by shaping different theories of intelligence. Reciprocalfilial piety beliefs were found to facilitate an incremental view ofintelligence, which in turn contributes to students' academic achievement.Authoritarian filial piety beliefs were shown to be associated with an entityview of intelligence, which consequently deteriorates students' academicachievement. Cultural views of motivational processes can shed light on howmotivational beliefs are developed as a product of cultural", metadata={'PMID': '24990635', 'source': 'https://pubmed.ncbi.nlm.nih.gov/24990635'})

In [9]:
len(splitted_documents)

227601

In [10]:
len(tokenizer.encode(splitted_documents[35].page_content))

163

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/e5-base-v2",
    model_kwargs={'device':'cpu'}, # Pass the model configuration options
    encode_kwargs={'normalize_embeddings': False} # Pass the encoding options
)

c:\Users\reda\Desktop\NLPT\Project_Medical-INLPT-WS2023\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Save the embeddings to csv file


In [79]:
from langchain.vectorstores import OpenSearchVectorSearch

auth = ('admin', '!akjdaDsdoij!oijadSsajd123120938') 

db = OpenSearchVectorSearch.from_documents(
    splitted_documents, embeddings, opensearch_url="http://localhost:9200", bulk_size=250000 , use_ssl = True, verify_certs = False, http_auth = auth
)

c:\Users\reda\Desktop\NLPT\Project_Medical-INLPT-WS2023\venv\lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\reda\Desktop\NLPT\Project_Medical-INLPT-WS2023\venv\lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\reda\Desktop\NLPT\Project_Medical-INLPT-WS2023\venv\lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.htm

ConnectionTimeout: ConnectionTimeout caused by - ReadTimeoutError(HTTPSConnectionPool(host='localhost', port=9200): Read timed out. (read timeout=10))

In [1]:
question = "What is cheesemaking?"
searchDocs = db.similarity_search(question)

NameError: name 'db' is not defined

In [ ]:
question = "What is cheesemaking?"
searchDocs = db.similarity_search(question, k= 2)

In [ ]:
searchDocs

[Document(page_content='CASK pathogenic variant and in a male who is hemizygousfor a CASK pathogenic variant on molecular genetic testing. Rarely, affectedmales have a mosaic pathogenic variant. MANAGEMENT: Treatment of manifestations:Treatment is symptomatic and includes standard management of developmental delayand intellectual disability issues; medication for seizures; nutritional support;use of physiotherapy; and treatment of abnormal vision or hearing loss. GENETICCOUNSELING: CASK disorders are inherited in an X-linked manner. Risk to thefamily members of a proband with a CASK disorder depends on the phenotype (i.e.,MICPCH or XLID +/- nystagmus) in the proband.', metadata={'PMID': '24278995', 'source': 'https://pubmed.ncbi.nlm.nih.gov/24278995'}),
 Document(page_content='with a CASK disorder depends on the phenotype (i.e.,MICPCH or XLID +/- nystagmus) in the proband. MICPCH. Most affected females andmales represent simplex cases (i.e., the only affected family member) and havethe

In [11]:
from opensearchpy import OpenSearch

In [12]:
#Initialize connection to opensearch
host = 'localhost'
port = 9200
auth = ('admin', '!akjdaDsdoij!oijadSsajd123120938') 

client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = False,
    timeout=100
)
#check status
print(client.info())

{'name': 'opensearch-node1', 'cluster_name': 'opensearch-cluster', 'cluster_uuid': 'VXIaLkT5T3WKA_hnmLqHdg', 'version': {'distribution': 'opensearch', 'number': '2.12.0', 'build_type': 'tar', 'build_hash': '2c355ce1a427e4a528778d4054436b5c4b756221', 'build_date': '2024-02-20T02:18:49.874618333Z', 'build_snapshot': False, 'lucene_version': '9.9.2', 'minimum_wire_compatibility_version': '7.10.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'The OpenSearch Project: https://opensearch.org/'}


c:\Users\reda\Desktop\NLPT\Project_Medical-INLPT-WS2023\venv\lib\site-packages\opensearchpy\connection\http_urllib3.py:208: UserWarning: Connecting to https://localhost:9200 using SSL with verify_certs=False is insecure.
  warnings.warn(
c:\Users\reda\Desktop\NLPT\Project_Medical-INLPT-WS2023\venv\lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [14]:
knn_search_body = {
    "size": 5,  # Number of nearest neighbors to retrieve
    "query": {
        "knn": {
            "vector": {
                "vector": embeddings,
                "k": 1  # Number of nearest neighbors to retrieve
            }
        }
    }
}

# Execute the search
response = client.search(index="deae3868b35644fa8893288e34be84bf", body=knn_search_body)

SerializationError: ({'size': 5, 'query': {'knn': {'vector': {'vector': HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
  (2): Normalize()
), model_name='intfloat/e5-base-v2', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': False}, multi_process=False, show_progress=False), 'k': 1}}}}, TypeError("Unable to serialize HuggingFaceEmbeddings(client=SentenceTransformer(\n  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel \n  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens':

In [ ]:
response

In [ ]:
# Query Transformation RAG system
from langchain.chat_models import ChatOpenAI
from langchain. retrievers.multi_query import MultiQueryRetriever

question = "What is cheesemaking?"
llm = ChatOpenAI(tempreture=0)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever = response.as_retriever(), llm=llm)